# 1D Model for a Stratified Storage Tank

The simulation of a stratified thermal storage tank will be based upon the model proposed by ____

The model is based on a 1D differential equation

$$
\begin{align*}
 T_{k+1,i} = & F_i(T_k, \dot{Q}_k, \dot{m}_k, \Delta k, \phi, T^{in}_k) \\ \\

 = & T_{k,i}+ (\alpha \frac{T_{k,i+1}+T_{k,i-1}-2T_{k,i}}{\Delta z^2_i} + \beta_i(T_\infty-T_{k,i}) \\ \\

 &+ \frac{\phi}{\Delta z_i}\sum_{l=0}^{i} \dot{m}_{k,l} \frac{(T^{in}_{k,l} - T_{k,i}) S (T^{in}_{k,l} - T_{k,i})}{\sum_{j=l}^{N} S (T^{in}_{k,l} - T_{k,j})} \\ \\

 &+ \frac{\phi}{\Delta z_i}\sum_{l=0}^{i} \dot{m}_{k,l} \frac{(T_{k,i} - T^{in}_{k,l}) S (T_{k,i} - T^{in}_{k,l})}{\sum_{j=0}^{l} S (T_{k,j} - T^{in}_{k,l})} \\ \\

 &+ \sum_{l=0}^{i} \frac{\lambda_i \dot{Q}_{k,l}}{\Delta z_i} \frac{S(T_{k,l}-T_{k,i})}{\sum_{j=l}^{N}S(T_{k,l}-T_{k,j})} \\ \\

 &+ \sum_{l=i}^{N} \frac{\lambda_i \dot{Q}_{k,l}}{\Delta z_i} \frac{S(T_{k,l}-T_{k,i})}{\sum_{j=0}^{i}S(T_{k,l}-T_{k,j})}) \Delta t \\ \\

&+ \theta_{i,(i-1)} \frac{1}{\mu} log(e^0+e^{\mu(T_{k,(i-1)}-T_{k,i})}) \\ \\

&- \theta_{i,(i+1)} \frac{1}{\mu} log(e^0+e^{\mu (T_{k,i}-T_{k,(i+1)})}) \\ \\
 \end{align*}
 
$$

where \
$\beta_i= \frac{P_i k_i}{\rho c_p A_i}$ \
$\lambda_i = \frac{1}{\rho c_p A_i} $ \
$\phi = \frac{1}{\rho A_i}$ \
$\theta_{i,(i-1)}= \frac{A_{i-1} \Delta z_{i-1}}{A_i \Delta z_i + A_{i-1} \Delta z_{i-1}} \in [0,1]$ \
$S(T_1 - T_2) = \frac{1}{1+e^{-\mu(T_1-T_2)}}$\
\
and \
$i$ tank layer \
$k$ time step \
$\Delta t$ lenght of time step \
$\mu$ sharpness of the logistic function (step function if high) \

- 𝛼: fluid diffusivity [m²/s]
- 𝛽𝑖: coefficient of heat losses of the inner layers
- 𝛽1: coefficient of heat losses of the bottom layer
- 𝛽𝑁: coefficient of heat losses of the top layer
- 𝜆: coefficient of the input heat
- 𝜙: coefficient of the input flow

## Simple model without buoyancy

The temperature of a layer of the tank will be determined by 4 main phenomena: the diffusion between the layers due to a gradient in the temperatures/pressures along the fluid in the tank, the heat losses to the environment through the surface of the tank, the indirect insertion or extraction of heat to the tank through heat exchange and the direct insertion or extraction of mass streams.


<img src="images/basic.png" alt="Alt Text" width="600">

### Model 0: [α] only diffusion within layers 
The thermal diffusivity of the fluid α is the rate at which heat spreads through a material over time, per (cross-secional) unit area.
It is assumed to be constant and for water it is

$
\alpha = \frac{k}{\rho \cdot c_p} = 0.143\cdot 10^{-6} \frac{m^2}{s}
$

Where:

$\alpha$ is the heat diffusivity coefficient of the fluid.[m²/s]\
$k$ is the thermal conductivity of the fluid. [W/mK]\
$\rho$ is the density of the fluid. \
$c_p$ is the specific heat capacity of the fluid.

Heat transfer equation for the diffusion within the layers i at times k

$$
\begin{align*}
\frac{\delta T}{\delta t} = & \alpha \nabla^2 T \\
= & \alpha  \frac{\delta^2 T}{\delta z^2}
\end{align*}
$$

#### Step 1: Discretization (Forward Euler Method)
- Time (k): approximate using simple differences

$$
\begin{align*}
\frac{\delta T}{\delta t} &\approx  \frac{\Delta T}{\Delta t}  \\
&\approx \frac{T_{k+1,i} - T_{k,i}}{\Delta t}
\end{align*}
$$


- Space (i): approximate using central differences

$$
\begin{align*}
\frac{\delta^2 T}{\delta z^2} 
&\approx\frac{T_{k,i+1}-2T_{k,i}+T_{k,i-1}}{(\Delta z_i)^2}
\end{align*}
$$

Which leads to the equation when replacing the discretization equations in the original heat transfer equation

$$
\begin{align*}
\frac{T_{k+1,i} - T_{k,i}}{\Delta t} = \alpha (\frac{T_{k,i+1}-2T_{k,i}+T_{k,i-1}}{(\Delta z_i)^2})
\end{align*}
$$

Reorganized to determine the temperature of a layer in the next time step:

$$
\begin{align*}
T_{k+1,i} = T_{k,i} + \alpha \cdot (\frac{T_{k,i+1}-2T_{k,i}+T_{k,i-1}}{(\Delta z_i)^2}) \cdot \Delta t


\end{align*}
$$

#### Step 2: stability check

Define the parameters. Depending on the chosen time step Δt and spatial step Δz the numerical solution will be more or less accurate. Usually the stability considerations lead to a constraint on the maximum allowable time step given by the Courant-Friedrichs-Lewy (CFL) condition (see equation below), and if the condition is met, then the simulation will not be performed. The time step must be reduced until the condition is met and the simulation can run.
$$
\begin{align*}
\Delta t \leq \frac{(\Delta z)^2}{2 \alpha}
\end{align*}
$$

#### Step 3: Simulation (HeatDistributionVector_model)
Through an iteration over time of the vector T, which contains the temperature distribution in the tank, the temperatures of each layer of the tank are updated after each time step.
Here several assumptions were made

##### Assumptions for simplification: 
- The amount of layers is defined by the amount of elements in the initial vector for the tank temperature
- Δz is constant and depends on the height of the tank and of the lenght of the vector T (number of layers)

##### Boundary conditions: 
since the equation requires the previous and the next layers temperature, we have to meet assumptions on the bottom and on the top, where the "next" or the "previous" do not exist, meaning that no heat transfer through diffusion is possible at this "cross-sections". This could be translated into a non-existant temperature gradient (adiabatic boundary condition for heat conduction) between the boundaries and the inexistant layers: 
- Bottom of the tank (i=0, first entry). The "previous" (non existant) layer is assumed to have the same temperature as the bottom layer. This way the gradient in the bottom of the tank is cero and no heat is transferred.
- Top of the tank (i=-1, last entry). The "next" (non existant) layer is assumed to have the same temperature as the top layer. This way the gradient in the bottom of the tank is cero and no heat is transferred. 

##### Plots
- Plot over the height of the tank for the different time steps
    - The x-axis displays the full height of the tank, and the y-axis the temperature. The y's are positioned in the middle of the layers.
- Plot over time for the different layers of the tank


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

############### Definitons for the model
class HeatDistributionVector_model0:
    def __init__(self, alpha, z, T_initial, dt):
        self.alpha = alpha                           # heat diffusivity
        self.z = z                                   # height of the tank
        self.num_layers = len(T_initial)             # number of layers (steps in space)
        self.dt = dt                                 # step size in time (time step)
        self.T_initial = np.array(T_initial)         # initial state of the temperatures along the tank [list] !!! Same lenght as num_layers
        
        self.dz = z / self.num_layers                # step size in space (delta z, height of a layer) is total height/number of layers, num_layers = how big the initial temperature vector is
        # Create a 
        self.heights = [i * self.dz + self.dz/2 for i in range(len(T_initial))]     # list representing the height of the tank for plotting the temperatures in the middle of each layer
  


 # definition of the solver for the temperature vector in the next time step       
    def vector_solve(self, num_steps):
        T_old = np.copy(self.T_initial)
        results = [T_old.copy()]                    # Store initial temperature array

        for _ in range(num_steps):

            T_new = np.copy(T_old)

            T_old_next = np.roll(T_old, -1)         # roll every i by -1 so that the "next" i is selected
            T_old_prev = np.roll(T_old, 1)          # roll every i by 1 so that the "previous" i is selected
            
            # Apply heat transfer equation for model 0
            T_new = (T_old 
                     + ((self.alpha) * (T_old_next - (2*T_old) + T_old_prev) / (self.dz**2)) * self.dt)
            
            # Boundary conditions
            T_new[0] = (T_old[0]
                         + ((self.alpha) * (T_old[1] - (2*T_old[0]) + T_old[0]) / (self.dz**2)) * self.dt)       # assuming no heat exchange in the boundary, the tmeperature "outside" of the tank (T_old_prev of first entry) would be the same as inside of the tank (T[0])
            T_new[-1] = (T_old[-1]
                          + ((self.alpha) * (T_old[-1] - (2*T_old[-1]) + T_old[-2]) / (self.dz**2)) * self.dt)   # assuming no heat exchange in the boundary, the tmeperature "outside" of the tank (T_old_next of last entry) would be the same as inside of the tank T[-1]

            T_old = np.copy(T_new) # return the new temperature as old temperature for the next iteration

            results.append(T_old.copy())            # Store the updated temperature array fo later plot

        return T_old, results
    
 # check the stability of the model with the selected dt
    def stability_check(self):
        # check if the time step dt is small enough with CFL condition: dt <= (dz^2) / (2 * alpha)
        cfl_dt_max = (self.dz ** 2) / (2 * self.alpha)
        if self.dt > cfl_dt_max:
            print(f"Warning: Time step size dt {self.dt} exceeds CFL stability limit ({cfl_dt_max}).")
            sc = 1
        else:
            sc = 0
        return sc
    

# Plot the layer temperatures over the height for different times
def plot_results_height(results, heights):
    # Create traces for each time step
    traces = []
    for i, temp_array in enumerate(results):            # results in [time x layers] -> enumerate gives all values of one row for one index (column) -> all layers (row) at one time
        time_passed = dt * i                            # Calculate the time passed
        trace = go.Scatter(
        x=heights,                                      # X axis is the height of each layer
            y=temp_array,                               # Y axis is the temperatures in each layer
            mode='lines',
            name=f'Time Passed: {time_passed} seconds'  # Use calculated time in legend
        )
        traces.append(trace)

    # Create the plot layout
    layout = go.Layout(
        title='Temperature Distribution Over Tank Height and Over Time',
        xaxis=dict(title='Height', range=[0, z]),
        yaxis=dict(title='Temperature'),
    )

    # Create the figure and plot it
    fig = go.Figure(data=traces, layout=layout)
    fig.show()

# Plot the layer temperatures over time
def plot_results_time(results, heights, z):
    # Create traces for each layer
    traces = []
    for i, temp_array in enumerate(np.array(results).T):  # Transpose the results array to iterate over layers -> [layers x time] -> row is the temperature over time of one sensor
        trace = go.Scatter(
            x=np.arange(len(temp_array)),                 # X axis is the timeline (Time steps)
            y=temp_array,                                 # Y axis is the temperature of all layers in each time step    
            mode='lines',
            name=f'Layer {i}'                               # Layer number
        )
        traces.append(trace)

    # Create the plot layout
    layout = go.Layout(
        title='Temperature of Each Layer Over Time',
        xaxis=dict(title='Time Step'),
        yaxis=dict(title='Temperature'),
    )

    # Create the figure and plot it
    fig = go.Figure(data=traces, layout=layout)
    fig.show()

############### 
            
# define variables  
    # lenght of the selected T_zero defines the number of layers    
T_1 = np.array([10, 10, 10, 10, 10, 90, 90, 90, 90, 90]) # initial temperature in the tank for each layer
T_2 = [10, 20, 30, 60, 50, 90, 30, 0, 15, 20]
T_3 = [100, 80, 50, 60, 20, 50, 10, 20, 50, 80]
T_4 = np.array([100, 90, 80 ,70, 60, 50, 40, 30, 20, 10])
T_5 = [100, 10, 80 ,200, 60, 70, -10, 30, -10, 10]

T_zero = T_2
alpha = 1#0.000000146                # heat diffusivity [0.146 x 10-6 m2/s]
z = 2                      # height of the tank [m]
dt = 0.02                      # length of the time steps  
num_steps = 20             # number of time steps to be made



tank_vector = HeatDistributionVector_model0(alpha, z, T_zero, dt)
stability = tank_vector.stability_check()

if (stability == 0):
    # Solve for the temperatures
    final_temperature, results = tank_vector.vector_solve(num_steps)
    # Plot the results
    plot_results_height(results, tank_vector.heights)
    plot_results_time(results, tank_vector.heights, tank_vector.z)



In [4]:
# FAILED NON-VECTOR APPROACH
"""import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

# dz is constant

class HeatDistribution1:
    def __init__(self, alpha, z, num_layers, T_initial):
        self.alpha = alpha                # heat diffusivity
        self.z = z                        # height of the tank
        self.num_layers = num_layers      # number of layers (steps in space)
        #self.dt = dt                     # step size in time (time step)
        self.T_initial = T_initial        # initial state of the temperatures along the tank [list] !!! Same lenght as num_layers
        

        self.dz = z / num_layers          # step size in space (delta z, height of a layer)

        # Create a list representing the height of the tank for plotting
        self.heights = [i * self.dz for i in range(num_layers)]

        


    def solve(self, num_steps, dt):     # time step numbers and size dt
        temp = np.copy(self.T_initial)          # initialize temperature 


        # iterate over time (time steps)
        for k in range(num_steps):
            # save old temp (either initial or calculated) as new temp
            new_temp = np.copy(temp)

            # update boundary conditions by assuming next closest temperature in the tank "on both sides" (outside of the tank)
            new_temp[0] = temp[0] + self.alpha * ((temp[1] - 2*temp[0] + temp[1]) / self.dz**2) * dt # boundary condition at the bottom of the tank (i=0), it takes the first temperature after boundary in the tank twice (i=1)
            new_temp[-1] = temp[-1] + self.alpha * ((temp[-2] - 2*temp[-1] + temp[-2]) / self.dz**2) * dt # boundary contidion at the top of the tank (i=-1), it takes the last temperature before boundary in the tank twice (i=-2)

            # iterate over space (layers)
            for i in range(1, self.num_layers - 1):
                new_temp[i] = temp[i] + self.alpha * ((temp[i+1] - 2*temp[i] + temp[i-1]) / self.dz**2) * dt

            temp = np.copy(new_temp)
                

             
        return temp



#if __name__ == "__main__":
### Definition of variables for the class
alpha = 0.001       # heat diffusivity
z = 10              # height of the tank
num_layers = 10     # number of layers of the tank

#T_initial1 = [10, 20, 30, 40, 50, 90, 50, 40, 15, 20] # initial temperature
T_initial2 = [10, 10, 10, 10, 10, 90, 90, 90, 90, 90]
T = T_initial2      # T to be passed on to the class


T_initial = [10, 20, 30, 60, 50, 90, 30, 0, 15, 20]
T_rand = [100, 80, 50, 60, 20, 50, 10, 20, 50, 80]
T_lin = [100, 90, 80 ,70, 60, 50, 40, 30, 20, 10]
T_x = [100, 10, 80 ,200, 60, 50, -10, 30, -100, 10]

### Definition of variables for the solver
num_steps = 1       # number of time steps to be made
dt = 1              # length of the time steps


# Create TankHeatDistribution instance
tank = HeatDistribution1(alpha, z, num_layers, T)


# Solve the model
temperature_distribution0 = tank.solve(0, dt)           #1s
temperature_distribution1 = tank.solve(1, dt)           #1s
temperature_distribution2 = tank.solve(3, dt)           #3s
temperature_distribution3 = tank.solve(10, dt)          #10s
temperature_distribution4 = tank.solve(30, dt)          #30s
temperature_distribution5 = tank.solve(40, dt)          #40s
temperature_distribution6 = tank.solve(50, dt)          #50s
temperature_distribution7 = tank.solve(60, dt)          #60s
temperature_distribution8 = tank.solve(80, dt)          #120s
temperature_distribution9 = tank.solve(500, dt)         #120s

# Create a figure
fig = go.Figure()
# Plot temperature data on primary y-axis

fig.add_trace(go.Scatter(x=tank.heights, y=T, mode='lines', name='initial'))
fig.add_trace(go.Scatter(x=tank.heights, y=temperature_distribution0, mode='lines', name='t0 = 0s'))
fig.add_trace(go.Scatter(x=tank.heights, y=temperature_distribution1, mode='lines', name='t1 = 1s'))
fig.add_trace(go.Scatter(x=tank.heights, y=temperature_distribution2, mode='lines', name='t2 = 3s'))
fig.add_trace(go.Scatter(x=tank.heights, y=temperature_distribution3, mode='lines', name='t3 = 10s'))
fig.add_trace(go.Scatter(x=tank.heights, y=temperature_distribution4, mode='lines', name='t4 = 30s'))
fig.add_trace(go.Scatter(x=tank.heights, y=temperature_distribution5, mode='lines', name='t5 = 40s'))
fig.add_trace(go.Scatter(x=tank.heights, y=temperature_distribution6, mode='lines', name='t6 = 50s'))
fig.add_trace(go.Scatter(x=tank.heights, y=temperature_distribution7, mode='lines', name='t7 = 60s'))
fig.add_trace(go.Scatter(x=tank.heights, y=temperature_distribution8, mode='lines', name='t8 = 80s'))
fig.add_trace(go.Scatter(x=tank.heights, y=temperature_distribution9, mode='lines', name='t8 = 100s'))

# Show plot
fig.show()
"""
    

'import numpy as np\nimport matplotlib.pyplot as plt\nimport plotly.graph_objects as go\nimport plotly.express as px\n\n# dz is constant\n\nclass HeatDistribution1:\n    def __init__(self, alpha, z, num_layers, T_initial):\n        self.alpha = alpha                # heat diffusivity\n        self.z = z                        # height of the tank\n        self.num_layers = num_layers      # number of layers (steps in space)\n        #self.dt = dt                     # step size in time (time step)\n        self.T_initial = T_initial        # initial state of the temperatures along the tank [list] !!! Same lenght as num_layers\n        \n\n        self.dz = z / num_layers          # step size in space (delta z, height of a layer)\n\n        # Create a list representing the height of the tank for plotting\n        self.heights = [i * self.dz for i in range(num_layers)]\n\n        \n\n\n    def solve(self, num_steps, dt):     # time step numbers and size dt\n        temp = np.copy(self.T_

### Model 1: [β] heat losses in the surface of the tank

The coefficient of heat loss β is derived from the energy balance of a system with heat losses to the ambient. It contains information about the geometry of the surface through which the heat will be transferred, the characteristics of the element of the balanced system (layer) and the heat loss properties of the isolation layer (U-Value or k), which describes the rate of heat loss per surface area and per unit temperature difference with the ambient. This coefficient remains constant throughout the inner layers of the tank (since the exposed surface area of the layers is constant). For the first (i=0) and last (i=-1) layers, the coefficient changes, since heat is also lost through the tank's bottom and top areas. For a cylindric tank where the ground and ceiling have the shape of the cross sectional area, following applies:


$
\beta_i = \frac{A_{s} \cdot k_i}{ V_i\cdot \rho \cdot c_p} =
\frac{P_i \cdot k_i}{\rho \cdot c_p \cdot A_i}
$

$
\beta_0 = \beta_i + \beta_{bottom} =\frac{P_i \cdot k_i}{\rho \cdot c_p \cdot A_i} + \frac{k_{bottom}}{\Delta z \cdot \rho \cdot c_p}
$

$
\beta_{-1} = \beta_i + \beta_{top} = \frac{P_i \cdot k_i}{\rho \cdot c_p \cdot A_i} + \frac{k_{top}}{\Delta z \cdot \rho \cdot c_p}
$

Where:

$\beta_i$ is the heat loss coefficient in the layer i [1/s]\
$k_i$ is the thermal conductivity of the wall in the layer i [W/(m²K)].\
$\rho$ is the density of the fluid [kg/m³]\
$c_p$ is the specific heat capacity of the fluid [J/kgK]\
$A_s$ is the exposed surface area of the tank [m²]\
$\Delta z$ is the height of the layer [m]\
$V_i$ is the volumen of the layer [m³]\
$P_i$ is the cross-sectional perimeter of the layer [m]\
$A_i$ is the cross-sectional area of the layer [m²]




Heat transfer equation for the diffusion within the layers i at times k regarding heat losses will be extended to include β

$$
\begin{align*}
\frac{\delta T_i}{\delta t} = & \alpha  \frac{\delta^2 T}{\delta z^2} + \frac{P_i k_i}{\rho  c_p  A_i}(T_\infty-T_i) \\
= & \alpha  \frac{\delta^2 T}{\delta z^2} + \beta_i(T_\infty-T_i) \\ 

\end{align*}
$$

resulting in the extended equation after applying the discretization step from Model 0

$$
\begin{align*}
T_{k+1,i} = T_{k,i} + (\alpha (\frac{T_{k,i+1}-2T_{k,i}+T_{k,i-1}}{(\Delta z_i)^2}) + \beta_i(T_\infty-T_i))\cdot \Delta t


\end{align*}
$$

The boundary conditions that were applied in Model 0 (no heat losses, only diffusion between inner layers) will be extended, so that the heat losses occuring in the surface of the tank can be depicted. For this, the betas in the layers have to be well defined:
##### Assumptions for simplification: 
- The amount of layers is defined by the amount of elements in the initial vector for the tank temperature
- Δz is constant and depends on the height of the tank and of the lenght of the vector T (number of layers)
- The area of the top and the bottom of the tank is the same as the cross-sectional area (circle)

##### Boundary conditions: 
The previous boundary conditions in Model 0 (heat diffussivity in the fluid) has to be extended to account for the heat losses at the bottom and top surface of the tank. The heat losses are accounted by the heat loss coefficienta $\beta_i$, and it has to be calculated and defined for each boundary (top and bottom): 

- Bottom of the tank (i=0, first entry). 
    - The "previous" (non existant) layer is assumed to have the same temperature as the bottom layer. This way the gradient in the bottom of the tank is cero and no heat is transferred due to diffusion.
    - The heat losses are accounted by the heat loss coefficient. Besides the heat loss through the Mantel of the tank (inlcuded in the normal model for the inner layers), an extra Heat loss throught the bottom of the tank $\beta_{bottom}$ has to be added to take into account the characteristics of the bottom of the physical tank and thus be able to calculate the heat lost through this surface to the ambient as well.
- Top of the tank (i=-1, last entry). 
    - The "next" (non existant) layer is assumed to have the same temperature as the top layer. This way the gradient in the bottom of the tank is cero and no heat is transferred due to diffusion. 
    - A coefficient $\beta_{top}$ will be defined taking into account the characteristics of the top of the physical tank and thus be able to calculate the heat lost through this surface to the ambient. This will be added to the existing losses through the Mantel of the inner layer.



In [2]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

############### Definitons for the model
class HeatDistributionVector_model1:
    def __init__(self, alpha, beta_i, beta_bottom, beta_top, z, T_a, T_initial, dt):
        self.alpha = alpha                           # heat diffusivity
        self.beta_i = beta_i                         # heat loss coefficient to the ambient in the inner layers
        self.beta_bottom = beta_bottom               # heat loss coefficient to the ambient at the bottom layer (i=0)
        self.beta_top = beta_top                     # heat loss coefficient to the ambient at the top layer (i=-1)
        
        self.z = z                                   # height of the tank
        self.num_layers = len(T_initial)             # number of layers (steps in space)
        self.dt = dt                                 # step size in time (time step)

        self.T_initial = np.array(T_initial)         # initial state of the temperatures along the tank [list] !!! Same lenght as num_layers
        self.T_a = T_a                                  # ambient temperature outside of the tank

        
        self.dz = z / self.num_layers                # step size in space (delta z, height of a layer) is total height/number of layers, num_layers = how big the initial temperature vector is
        # Create a 
        self.heights = [i * self.dz + self.dz/2 for i in range(len(T_initial))]     # list representing the height of the tank for plotting the temperatures in the middle of each layer
  


 # definition of the solver for the temperature vector in the next time step       
    def vector_solve(self, num_steps):
        T_old = np.copy(self.T_initial)
        results = [T_old.copy()]                    # Store initial temperature array

        for _ in range(num_steps):

            T_new = np.copy(T_old)

            T_old_next = np.roll(T_old, -1)         # roll every i by -1 so that the "next" i is selected
            T_old_prev = np.roll(T_old, 1)          # roll every i by 1 so that the "previous" i is selected
            
            # Apply heat transfer equation for model 0
            T_new = (T_old
                      + (((self.alpha) * (T_old_next - (2*T_old) + T_old_prev) / (self.dz**2))
                      + (self.beta_i * (self.T_a - T_old))
                      )* self.dt)
            
            # Boundary conditions
            T_new[0] = (T_old[0]
                         + (((self.alpha) * (T_old[1] - (2*T_old[0]) + T_old[0]) / (self.dz**2))
                         + ((self.beta_i + self.beta_bottom) * (self.T_a - T_old[0]))  
                         )* self.dt)        # heat loss through sides of the tank (beta_i) and through the floor (beta_bottom)
            T_new[-1] = (T_old[-1]
                          + (((self.alpha) * (T_old[-1] - (2*T_old[-1]) + T_old[-2]) / (self.dz**2))
                          + ((self.beta_i + self.beta_top)* (self.T_a - T_old[-1])) 
                          )* self.dt)       # heat loss through sides of the tank (beta_i) and through ceiling (beta_top)

            T_old = np.copy(T_new) # return the new temperature as old temperature for the next iteration

            results.append(T_old.copy())            # Store the updated temperature array fo later plot

        return T_old, results
    
 # check the stability of the model with the selected dt
    def stability_check(self):
        # check if the time step dt is small enough with CFL condition: dt <= (dz^2) / (2 * alpha)
        cfl_dt_max = (self.dz ** 2) / (2 * self.alpha)
        if self.dt > cfl_dt_max:
            print(f"Warning: Time step size dt {self.dt} exceeds CFL stability limit ({cfl_dt_max}).")
            sc = 1
        else:
            sc = 0
        return sc
    

# Plot the layer temperatures over the height for different times
def plot_results_height(results, heights):
    # Create traces for each time step
    traces = []
    for i, temp_array in enumerate(results):            # results in [time x layers] -> enumerate gives all values of one row for one index (column) -> all layers (row) at one time
        time_passed = dt * i                            # Calculate the time passed
        trace = go.Scatter(
            x=heights,                                      # X axis is the height of each layer
            y=temp_array,                               # Y axis is the temperatures in each layer
            mode='lines',
            name=f'Time Passed: {time_passed} seconds'   # Use calculated time in legend
        )
        traces.append(trace)

    # Create the plot layout
    layout = go.Layout(
        title='Temperature Distribution Over Tank Height and Over Time',
        xaxis=dict(title='Height', range=[0, z]),
        yaxis=dict(title='Temperature'),
    )

    # Create the figure and plot it
    fig = go.Figure(data=traces, layout=layout)
    fig.show()

# Plot the layer temperatures over time
def plot_results_time(results, heights, z):
    # Create traces for each layer
    traces = []
    for i, temp_array in enumerate(np.array(results).T):  # Transpose the results array to iterate over layers -> [layers x time] -> row is the temperature over time of one sensor
        time_passed = dt * i                            # Calculate the time passed
        trace = go.Scatter(
            x=np.arange(len(temp_array)),                 # X axis is the timeline (Time steps)
            y=temp_array,                                 # Y axis is the temperature of all layers in each time step    
            mode='lines',
            name=f'Layer {i}'                               # Layer number
        )
        traces.append(trace)

    # Create the plot layout
    layout = go.Layout(
        title='Temperature of Each Layer Over Time',
        xaxis=dict(title='Time Step'),
        yaxis=dict(title='Temperature'),
    )

    # Create the figure and plot it
    fig = go.Figure(data=traces, layout=layout)
    fig.show()

############### 
            
# initial temperature in the tank for each layer
# lenght of the selected T_zero defines the number of layers    
T_1 = np.array([10, 10, 10, 10, 10, 90, 90, 90, 90, 90]) 
T_2 = np.array([10, 20, 30, 60, 50, 90, 30, 0, 15, 20])
T_3 = [100, 80, 50, 60, 20, 50, 10, 20, 50, 80]
T_4 = np.array([100, 90, 80 ,70, 60, 50, 40, 30, 20, 10])
T_5 = [100, 10, 80 ,200, 60, 70, -10, 30, -10, 10]
T_6 = np.array([100, 100, 100, 100, 100, 100, 100, 100, 100, 100]) 
T_7 = np.array([65, 65, 65, 65, 65, 65, 65, 65, 65, 65]) # standard initial temperature

T_a = 20                            # ambient temperature

# tank description
T_zero = T_2                        # initial temperature vector
z = 2.099                               # height of the tank [m]
dz = z / len(T_zero)                # height of the section (layer)
d = 0.79                             # diameter of the cross section [m]
P_i = np.pi * d                     # cross-sectional perimeter [m]
A_i = np.pi * (d/2)**2              # cross-sectional area [m²]

# material properties
alpha = 0.000000146                 # heat diffusivity of the fluid [m²/s]
rho = 998                           # density of the fluid [kg/m³]
cp =  4186                          # heat capacity of the fluid [J/kgK]
k_i = 0.5                           # thermal condunctance of the wall [W/m²K]


dt = 60                             # length of the time steps [s]
num_steps = 100                    # number of time steps to be made


# Beta calculations
beta_i = (P_i*k_i)/(rho * cp * A_i) # coefficient of heat losses through the wall of the tank [1/s]
beta_top = (k_i/(rho * cp * dz))    # coefficient of heat losses through the ceiling of the tank [1/s]
beta_bottom = (k_i/(rho * cp * dz)) # coefficient of heat losses through the ground of the tank [1/s]

print("beta_i", beta_i)
print("beta_top", beta_top)
print("beta_bottom", beta_bottom)

qloss = 138.75
Uwert = (qloss)/((P_i*z + 2*A_i)*(65-20))
Uwert2 = (qloss)/((P_i*z + A_i)*(65-20))
print("U-wert", Uwert)
print("U-wert2", Uwert2)
print("defined k_i", k_i)

tank_vector = HeatDistributionVector_model1(alpha, beta_i, beta_bottom, beta_top, z, T_a, T_zero, dt)
stability = tank_vector.stability_check()

if (stability == 0):
    # Solve for the temperatures
    final_temperature, results = tank_vector.vector_solve(num_steps)
    # Plot the results
    plot_results_height(results, tank_vector.heights)
    plot_results_time(results, tank_vector.heights, tank_vector.z)


"""
tank_vector0 = HeatDistributionVector_model0(alpha, z, T_zero, dt)
stability = tank_vector0.stability_check()

if (stability == 0):
    # Solve for the temperatures
    final_temperature, results = tank_vector0.vector_solve(num_steps)
    # Plot the results
    plot_results_height(results, tank_vector0.heights)
    plot_results_time(results, tank_vector0.heights, tank_vector.z)

"""



beta_i 6.060007184986919e-07
beta_top 5.702007713363108e-07
beta_bottom 5.702007713363108e-07
U-wert 0.49813500878057776
U-wert2 0.5409748364462272
defined k_i 0.5


'\ntank_vector0 = HeatDistributionVector_model0(alpha, z, T_zero, dt)\nstability = tank_vector0.stability_check()\n\nif (stability == 0):\n    # Solve for the temperatures\n    final_temperature, results = tank_vector0.vector_solve(num_steps)\n    # Plot the results\n    plot_results_height(results, tank_vector0.heights)\n    plot_results_time(results, tank_vector0.heights, tank_vector.z)\n\n'

# Model 2: (in)direct heat exchange

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

############### Definitons for the model
class HeatDistributionVector_model2:
    def __init__(self, alpha, beta_i, beta_bottom, beta_top, lambda_i, phi_i, z, T_a, T_initial, dt, Qdot, mdot, Tm):
        self.alpha = alpha                           # heat diffusivity
        self.beta_i = beta_i                         # heat loss coefficient to the ambient in the inner layers
        self.beta_bottom = beta_bottom               # heat loss coefficient to the ambient at the bottom layer (i=0)
        self.beta_top = beta_top                     # heat loss coefficient to the ambient at the top layer (i=-1)
        self.lambda_i = lambda_i                     # coefficient of the input heat
        self.phi_i = phi_i                           # coefficient of the input flow/stream
        
        self.z = z                                   # height of the tank
        self.num_layers = len(T_initial)             # number of layers (steps in space)
        self.dt = dt                                 # step size in time (time step)

        self.T_initial = np.array(T_initial)         # initial state of the temperatures along the tank [list] !!! Same lenght as num_layers
        self.T_a = T_a                               # ambient temperature outside of the tank
        self.Qdot= Qdot                              # vector conaining the Q_i of each layer
        self.mdot = mdot                             # vector conaining the streams flowing into /out of the tank mdot_i of each layer
        self.Tm = Tm                                 # vector containing the temperatures of the streams flowing in/out of the tank (each mdot_i has a Tm_i)

        
        self.dz = z / self.num_layers                # step size in space (delta z, height of a layer) is total height/number of layers, num_layers = how big the initial temperature vector is
        # Create a 
        self.heights = [i * self.dz + self.dz/2 for i in range(len(T_initial))]     # list representing the height of the tank for plotting the temperatures in the middle of each layer
  


 # definition of the solver for the temperature vector in the next time step       
    def vector_solve(self, num_steps):
        T_old = np.copy(self.T_initial)
        results = [T_old.copy()]                    # Store initial temperature array

        for _ in range(num_steps):

            T_new = np.copy(T_old)

            T_old_next = np.roll(T_old, -1)         # roll every i by -1 so that the "next" i is selected
            T_old_prev = np.roll(T_old, 1)          # roll every i by 1 so that the "previous" i is selected
            
            # Apply heat transfer equation for model 0
            T_new = (T_old
                      + (((self.alpha) * (T_old_next - (2*T_old) + T_old_prev) / (self.dz**2))  # diffusion between layers
                      + (self.beta_i * (self.T_a - T_old))                                      # losses to the ambient
                      + ((self.lambda_i/self.dz) * self.Qdot)                                   # indirect heat input (heat exchanger)
                      + ((self.phi_i/self.dz) * self.mdot * (self.Tm - T_old))                  # direct heat input (stream)
                      )* self.dt)
            
            # Boundary conditions
            T_new[0] = (T_old[0]
                         + (((self.alpha) * (T_old[1] - (2*T_old[0]) + T_old[0]) / (self.dz**2))
                         + ((self.beta_i + self.beta_bottom) * (self.T_a - T_old[0]))
                         + ((self.lambda_i/self.dz) * self.Qdot[0])
                         + ((self.phi_i/self.dz) * self.mdot[0] * (self.Tm[0] - T_old[0]))  
                         )* self.dt)        # heat loss through sides of the tank (beta_i) and through the floor (beta_bottom)
            
            T_new[-1] = (T_old[-1]
                          + (((self.alpha) * (T_old[-1] - (2*T_old[-1]) + T_old[-2]) / (self.dz**2))
                          + ((self.beta_i + self.beta_top)* (self.T_a - T_old[-1])) 
                          + ((self.lambda_i/self.dz) * self.Qdot[-1])
                          + ((self.phi_i/self.dz) * self.mdot[-1] * (self.Tm[-1] - T_old[-1]))
                          )* self.dt)       # heat loss through sides of the tank (beta_i) and through ceiling (beta_top)

            T_old = np.copy(T_new) # return the new temperature as old temperature for the next iteration

            results.append(T_old.copy())            # Store the updated temperature array fo later plot

        return T_old, results
     
 # check the stability of the model with the selected dt
    def stability_check(self):
        # check if the time step dt is small enough with CFL condition: dt <= (dz^2) / (2 * alpha)
        cfl_dt_max = (self.dz ** 2) / (2 * self.alpha)
        if self.dt > cfl_dt_max:
            print(f"Warning: Time step size dt {self.dt} exceeds CFL stability limit ({cfl_dt_max}).")
            sc = 1
        else:
            sc = 0
        return sc
    

# Plot the layer temperatures over the height for different times
def plot_results_height(results, heights):
    # Create traces for each time step
    traces = []
    for i, temp_array in enumerate(results):            # results in [time x layers] -> enumerate gives all values of one row for one index (column) -> all layers (row) at one time
        time_passed = dt * i                            # Calculate the time passed
        trace = go.Scatter(
            x=heights,                                      # X axis is the height of each layer
            y=temp_array,                               # Y axis is the temperatures in each layer
            mode='lines',
            name=f'Time Passed: {time_passed} seconds'   # Use calculated time in legend
        )
        traces.append(trace)

    # Create the plot layout
    layout = go.Layout(
        title='Temperature Distribution Over Tank Height and Over Time',
        xaxis=dict(title='Height', range=[0, z]),
        yaxis=dict(title='Temperature'),
    )

    # Create the figure and plot it
    fig = go.Figure(data=traces, layout=layout)
    fig.show()

# Plot the layer temperatures over time
def plot_results_time(results, heights, z):
    # Create traces for each layer
    traces = []
    for i, temp_array in enumerate(np.array(results).T):  # Transpose the results array to iterate over layers -> [layers x time] -> row is the temperature over time of one sensor
        time_passed = dt * i                            # Calculate the time passed
        trace = go.Scatter(
            x=np.arange(len(temp_array)),                 # X axis is the timeline (Time steps)
            y=temp_array,                                 # Y axis is the temperature of all layers in each time step    
            mode='lines',
            name=f'Layer {i}'                               # Layer number
        )
        traces.append(trace)

    # Create the plot layout
    layout = go.Layout(
        title='Temperature of Each Layer Over Time',
        xaxis=dict(title='Time Step'),
        yaxis=dict(title='Temperature'),
    )

    # Create the figure and plot it
    fig = go.Figure(data=traces, layout=layout)
    fig.show()



############### 
            
# initial temperature in the tank for each layer
# lenght of the selected T_zero defines the number of layers    
T_1 = np.array([10, 10, 10, 10, 10, 90, 90, 90, 90, 90]) 
T_2 = np.array([10, 20, 30, 60, 50, 90, 30, 0, 15, 20])
T_3 = [100, 80, 50, 60, 20, 50, 10, 20, 50, 80]
T_4 = np.array([100, 90, 80 ,70, 60, 50, 40, 30, 20, 10])
T_5 = [100, 10, 80 ,200, 60, 70, -10, 30, -10, 10]
T_6 = np.array([100, 100, 100, 100, 100, 100, 100, 100, 100, 100]) 
T_7 = np.array([65, 65, 65, 65, 65, 65, 65, 65, 65, 65]) # standard initial temperature
T_8 = np.full(10, 40)

T_a = 20                            # ambient temperature

# tank description
T_zero = T_8                        # initial temperature vector
z = 2.099                               # height of the tank [m]
dz = z / len(T_zero)                # height of the section (layer)
d = 0.79                             # diameter of the cross section [m]
P_i = np.pi * d                     # cross-sectional perimeter [m]
A_i = np.pi * (d/2)**2              # cross-sectional area [m²]


# material properties
alpha = 0.000000146                 # heat diffusivity of the fluid [m²/s]
rho = 998                           # density of the fluid [kg/m³]
cp =  4186                          # heat capacity of the fluid [J/kgK]
k_i = 0.5                           # thermal condunctance of the wall [W/m²K]

# indirect heat
lambda_i = (1/(A_i*rho*cp))         # coefficient of the input heat
Qdot0 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Qdot1 = np.array([0, 0, 0, 0, 0
                  
                  , 0, 0, 0, 0, 0])
Qdot = Qdot1

# direct heat
phi_i = (1/(A_i*rho))
mdot = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Tm = np.full(10,70)


dt = 60                             # length of the time steps [s]
num_steps = 10                    # number of time steps to be made


# losses: Beta calculations
beta_i = (P_i*k_i)/(rho * cp * A_i) # coefficient of heat losses through the wall of the tank [1/s]
beta_top = (k_i/(rho * cp * dz))    # coefficient of heat losses through the ceiling of the tank [1/s]
beta_bottom = (k_i/(rho * cp * dz)) # coefficient of heat losses through the ground of the tank [1/s]

print("beta_i", beta_i)
print("beta_top", beta_top)
print("beta_bottom", beta_bottom)

qloss = 138.75
Uwert = (qloss)/((P_i*z + 2*A_i)*(65-20))
Uwert2 = (qloss)/((P_i*z + A_i)*(65-20))
print("U-wert", Uwert)
print("U-wert2", Uwert2)
print("defined k_i", k_i)

tank_vector = HeatDistributionVector_model2(alpha, beta_i, beta_bottom, beta_top, lambda_i, phi_i, z, T_a, T_zero, dt, Qdot, mdot, Tm)
stability = tank_vector.stability_check()

if (stability == 0):
    # Solve for the temperatures
    final_temperature, results = tank_vector.vector_solve(num_steps)
    # Plot the results
    plot_results_height(results, tank_vector.heights)
    #plot_results_time(results, tank_vector.heights, tank_vector.z)



tank_vector1 = HeatDistributionVector_model1(alpha, beta_i, beta_bottom, beta_top, z, T_a, T_zero, dt)
stability = tank_vector1.stability_check()

if (stability == 0):
    # Solve for the temperatures
    final_temperature, results = tank_vector1.vector_solve(num_steps)
    # Plot the results
    plot_results_height(results, tank_vector1.heights)
    #plot_results_time(results, tank_vector1.heights, tank_vector1.z)



beta_i 6.060007184986919e-07
beta_top 5.702007713363108e-07
beta_bottom 5.702007713363108e-07
U-wert 0.49813500878057776
U-wert2 0.5409748364462272
defined k_i 0.5


In [7]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

import pandas as pd

# Create datetime index with 1-minute intervals starting from 01.01.2024 at 13:00 hours
start_time = pd.Timestamp('2024-01-01 13:00:00')
end_time = start_time + pd.Timedelta(minutes=100)
datetime_index = pd.date_range(start=start_time, end=end_time, freq='1Min')

# Create DataFrame with datetime index
df = pd.DataFrame(index=datetime_index)

# Initialize columns with NaN values
df['flow1'] = pd.Series([np.nan] * len(df), index=df.index)
df['temperature1'] = pd.Series([np.nan] * len(df), index=df.index)
df['flow2'] = pd.Series([np.nan] * len(df), index=df.index)
df['temperature2'] = pd.Series([np.nan] * len(df), index=df.index)
df['qdot1'] = pd.Series([np.nan] * len(df), index=df.index)

# Fill the DataFrame according to specified conditions
# Between 13:10 and 13:30, set flow1 = -0.2 and temperature1 = 10°C
df.loc['2024-01-01 13:10:00':'2024-01-01 13:30:00', 'flow1'] = -0.2
df.loc['2024-01-01 13:10:00':'2024-01-01 13:30:00', 'temperature1'] = 10

# Between 13:10 and 13:30, set flow2 = 0.2 and temperature2 = 60°C
df.loc['2024-01-01 13:10:00':'2024-01-01 13:30:00', 'flow2'] = 0.2
df.loc['2024-01-01 13:10:00':'2024-01-01 13:30:00', 'temperature2'] = 60

# Between 13:50 and 14:00, set qdot1 = 1000
df.loc['2024-01-01 13:50:00':'2024-01-01 14:00:00', 'qdot1'] = 1000

print(df)



                     flow1  temperature1  flow2  temperature2  qdot1
2024-01-01 13:00:00    NaN           NaN    NaN           NaN    NaN
2024-01-01 13:01:00    NaN           NaN    NaN           NaN    NaN
2024-01-01 13:02:00    NaN           NaN    NaN           NaN    NaN
2024-01-01 13:03:00    NaN           NaN    NaN           NaN    NaN
2024-01-01 13:04:00    NaN           NaN    NaN           NaN    NaN
...                    ...           ...    ...           ...    ...
2024-01-01 14:36:00    NaN           NaN    NaN           NaN    NaN
2024-01-01 14:37:00    NaN           NaN    NaN           NaN    NaN
2024-01-01 14:38:00    NaN           NaN    NaN           NaN    NaN
2024-01-01 14:39:00    NaN           NaN    NaN           NaN    NaN
2024-01-01 14:40:00    NaN           NaN    NaN           NaN    NaN

[101 rows x 5 columns]


In [8]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

############### Definitons for the model
class HeatDistributionVector_model2:
    def __init__(self, alpha, beta_i, beta_bottom, beta_top, lambda_i, phi_i, z, T_a, T_initial, dt, Qdot, mdot, Tm):
        self.alpha = alpha                           # heat diffusivity
        self.beta_i = beta_i                         # heat loss coefficient to the ambient in the inner layers
        self.beta_bottom = beta_bottom               # heat loss coefficient to the ambient at the bottom layer (i=0)
        self.beta_top = beta_top                     # heat loss coefficient to the ambient at the top layer (i=-1)
        self.lambda_i = lambda_i                     # coefficient of the input heat
        self.phi_i = phi_i                           # coefficient of the input flow/stream
        
        self.z = z                                   # height of the tank
        self.num_layers = len(T_initial)             # number of layers (steps in space)
        self.dt = dt                                 # step size in time (time step)

        self.T_initial = np.array(T_initial)         # initial state of the temperatures along the tank [list] !!! Same lenght as num_layers
        self.T_a = T_a                               # ambient temperature outside of the tank
        self.Qdot= Qdot                              # vector conaining the Q_i of each layer
        self.mdot = mdot                             # vector conaining the streams flowing into /out of the tank mdot_i of each layer
        self.Tm = Tm                                 # vector containing the temperatures of the streams flowing in/out of the tank (each mdot_i has a Tm_i)

        
        self.dz = z / self.num_layers                # step size in space (delta z, height of a layer) is total height/number of layers, num_layers = how big the initial temperature vector is
        # Create a 
        self.heights = [i * self.dz + self.dz/2 for i in range(len(T_initial))]     # list representing the height of the tank for plotting the temperatures in the middle of each layer
  


 # definition of the solver for the temperature vector in the next time step       
    def vector_solve(self, num_steps):
        T_old = np.copy(self.T_initial)
        results = [T_old.copy()]                    # Store initial temperature array

        for _ in range(num_steps):

            T_new = np.copy(T_old)

            T_old_next = np.roll(T_old, -1)         # roll every i by -1 so that the "next" i is selected
            T_old_prev = np.roll(T_old, 1)          # roll every i by 1 so that the "previous" i is selected
            
            # Apply heat transfer equation for model 0
            T_new = (T_old
                      + (((self.alpha) * (T_old_next - (2*T_old) + T_old_prev) / (self.dz**2))  # diffusion between layers
                      + (self.beta_i * (self.T_a - T_old))                                      # losses to the ambient
                      + ((self.lambda_i/self.dz) * self.Qdot)                                   # indirect heat input (heat exchanger)
                      + ((self.phi_i/self.dz) * self.mdot * (self.Tm - T_old))                  # direct heat input (stream)
                      )* self.dt)
            
            # Boundary conditions
            T_new[0] = (T_old[0]
                         + (((self.alpha) * (T_old[1] - (2*T_old[0]) + T_old[0]) / (self.dz**2))
                         + ((self.beta_i + self.beta_bottom) * (self.T_a - T_old[0]))
                         + ((self.lambda_i/self.dz) * self.Qdot[0])
                         + ((self.phi_i/self.dz) * self.mdot[0] * (self.Tm[0] - T_old[0]))  
                         )* self.dt)        # heat loss through sides of the tank (beta_i) and through the floor (beta_bottom)
            
            T_new[-1] = (T_old[-1]
                          + (((self.alpha) * (T_old[-1] - (2*T_old[-1]) + T_old[-2]) / (self.dz**2))
                          + ((self.beta_i + self.beta_top)* (self.T_a - T_old[-1])) 
                          + ((self.lambda_i/self.dz) * self.Qdot[-1])
                          + ((self.phi_i/self.dz) * self.mdot[-1] * (self.Tm[-1] - T_old[-1]))
                          )* self.dt)       # heat loss through sides of the tank (beta_i) and through ceiling (beta_top)

            T_old = np.copy(T_new) # return the new temperature as old temperature for the next iteration

            results.append(T_old.copy())            # Store the updated temperature array fo later plot

        return T_old, results
     
 # check the stability of the model with the selected dt
    def stability_check(self):
        # check if the time step dt is small enough with CFL condition: dt <= (dz^2) / (2 * alpha)
        cfl_dt_max = (self.dz ** 2) / (2 * self.alpha)
        if self.dt > cfl_dt_max:
            print(f"Warning: Time step size dt {self.dt} exceeds CFL stability limit ({cfl_dt_max}).")
            sc = 1
        else:
            sc = 0
        return sc
    

# Plot the layer temperatures over the height for different times
def plot_results_height(results, heights):
    # Create traces for each time step
    traces = []
    for i, temp_array in enumerate(results):            # results in [time x layers] -> enumerate gives all values of one row for one index (column) -> all layers (row) at one time
        time_passed = dt * i                            # Calculate the time passed
        trace = go.Scatter(
            x=heights,                                      # X axis is the height of each layer
            y=temp_array,                               # Y axis is the temperatures in each layer
            mode='lines',
            name=f'Time Passed: {time_passed} seconds'   # Use calculated time in legend
        )
        traces.append(trace)

    # Create the plot layout
    layout = go.Layout(
        title='Temperature Distribution Over Tank Height and Over Time',
        xaxis=dict(title='Height', range=[0, z]),
        yaxis=dict(title='Temperature'),
    )

    # Create the figure and plot it
    fig = go.Figure(data=traces, layout=layout)
    fig.show()

# Plot the layer temperatures over time
def plot_results_time(results, heights, z):
    # Create traces for each layer
    traces = []
    for i, temp_array in enumerate(np.array(results).T):  # Transpose the results array to iterate over layers -> [layers x time] -> row is the temperature over time of one sensor
        time_passed = dt * i                            # Calculate the time passed
        trace = go.Scatter(
            x=np.arange(len(temp_array)),                 # X axis is the timeline (Time steps)
            y=temp_array,                                 # Y axis is the temperature of all layers in each time step    
            mode='lines',
            name=f'Layer {i}'                               # Layer number
        )
        traces.append(trace)

    # Create the plot layout
    layout = go.Layout(
        title='Temperature of Each Layer Over Time',
        xaxis=dict(title='Time Step'),
        yaxis=dict(title='Temperature'),
    )

    # Create the figure and plot it
    fig = go.Figure(data=traces, layout=layout)
    fig.show()



############### 
            
# initial temperature in the tank for each layer
# lenght of the selected T_zero defines the number of layers    
T_1 = np.array([10, 10, 10, 10, 10, 90, 90, 90, 90, 90]) 
T_2 = np.array([10, 20, 30, 60, 50, 90, 30, 0, 15, 20])
T_3 = [100, 80, 50, 60, 20, 50, 10, 20, 50, 80]
T_4 = np.array([100, 90, 80 ,70, 60, 50, 40, 30, 20, 10])
T_5 = [100, 10, 80 ,200, 60, 70, -10, 30, -10, 10]
T_6 = np.array([100, 100, 100, 100, 100, 100, 100, 100, 100, 100]) 
T_7 = np.array([65, 65, 65, 65, 65, 65, 65, 65, 65, 65]) # standard initial temperature
T_8 = np.full(10, 40)

T_a = 20                            # ambient temperature

# tank description
T_zero = T_8                        # initial temperature vector
z = 2.099                               # height of the tank [m]
dz = z / len(T_zero)                # height of the section (layer)
d = 0.79                             # diameter of the cross section [m]
P_i = np.pi * d                     # cross-sectional perimeter [m]
A_i = np.pi * (d/2)**2              # cross-sectional area [m²]


# material properties
alpha = 0.000000146                 # heat diffusivity of the fluid [m²/s]
rho = 998                           # density of the fluid [kg/m³]
cp =  4186                          # heat capacity of the fluid [J/kgK]
k_i = 0.5                           # thermal condunctance of the wall [W/m²K]

# indirect heat
lambda_i = (1/(A_i*rho*cp))         # coefficient of the input heat
Qdot0 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Qdot1 = np.array([0, 0, 0, 0, 0
                  
                  , 0, 0, 0, 0, 0])
Qdot = Qdot1

# direct heat
phi_i = (1/(A_i*rho))
mdot = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Tm = np.full(10,70)


dt = 60                             # length of the time steps [s]
num_steps = 10                    # number of time steps to be made


# losses: Beta calculations
beta_i = (P_i*k_i)/(rho * cp * A_i) # coefficient of heat losses through the wall of the tank [1/s]
beta_top = (k_i/(rho * cp * dz))    # coefficient of heat losses through the ceiling of the tank [1/s]
beta_bottom = (k_i/(rho * cp * dz)) # coefficient of heat losses through the ground of the tank [1/s]

print("beta_i", beta_i)
print("beta_top", beta_top)
print("beta_bottom", beta_bottom)

qloss = 138.75
Uwert = (qloss)/((P_i*z + 2*A_i)*(65-20))
Uwert2 = (qloss)/((P_i*z + A_i)*(65-20))
print("U-wert", Uwert)
print("U-wert2", Uwert2)
print("defined k_i", k_i)

tank_vector = HeatDistributionVector_model2(alpha, beta_i, beta_bottom, beta_top, lambda_i, phi_i, z, T_a, T_zero, dt, Qdot, mdot, Tm)
stability = tank_vector.stability_check()

if (stability == 0):
    # Solve for the temperatures
    final_temperature, results = tank_vector.vector_solve(num_steps)
    # Plot the results
    plot_results_height(results, tank_vector.heights)
    #plot_results_time(results, tank_vector.heights, tank_vector.z)



tank_vector1 = HeatDistributionVector_model1(alpha, beta_i, beta_bottom, beta_top, z, T_a, T_zero, dt)
stability = tank_vector1.stability_check()

if (stability == 0):
    # Solve for the temperatures
    final_temperature, results = tank_vector1.vector_solve(num_steps)
    # Plot the results
    plot_results_height(results, tank_vector1.heights)
    #plot_results_time(results, tank_vector1.heights, tank_vector1.z)



beta_i 6.060007184986919e-07
beta_top 5.702007713363108e-07
beta_bottom 5.702007713363108e-07
U-wert 0.49813500878057776
U-wert2 0.5409748364462272
defined k_i 0.5
